In [1]:
import pandas as pd
import numpy as np

In [3]:
tourney_df = pd.read_csv('data/TourneyDetailedResults.csv')
teams_df = pd.read_csv('data/Teams.csv')
reg_season_df = pd.read_csv('data/RegularSeasonCompactResults.csv')
team_dict = dict(zip(teams_df['Team_Id'].values, teams_df['Team_Name'].values))
tourney_df['Wteam_name'] = tourney_df['Wteam'].map(team_dict)
tourney_df['Lteam_name'] = tourney_df['Lteam'].map(team_dict)

In [6]:
seasons = tourney_df['Season'].unique()
in_reg_season = dict(zip(seasons, [reg_season_df['Season'] == i for i in seasons]))
valid = reg_season_df['Wteam'] != 0

def features(id1, id2, season, flip):
    f1 = []
    f2 = []
    wins = reg_season_df.loc[valid & in_reg_season[season], 'Wteam'].value_counts()
    f1.append(wins[id1])
    f2.append(wins[id2])
    
    if flip:
        return f2+f1
    else:
        return f1+f2

In [9]:
Ngames = tourney_df.shape[0]
y = np.random.randint(2,size=Ngames)
X = np.asarray([features(tourney_df.loc[i, 'Wteam'], tourney_df.loc[i, 'Lteam'], tourney_df.loc[i, 'Season'], y[i]) for i in range(Ngames)])

In [11]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [12]:
from sklearn import svm
model = svm.SVC()
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
from sklearn import metrics
y_pred = model.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))

0.625368731563
